# Webapp: set up data stream, visualize, and equitize

- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
- https://dash.plotly.com/live-updates
- https://pbpython.com/interactive-dashboards.html#id6
- https://mybinder.org/ + https://github.com/echow1/trading_music
- (maybe) https://www.freecodecamp.org/news/how-to-create-auto-updating-data-visualizations-in-python-with-matplotlib-and-aws/
- https://kapernikov.com/ipywidgets-with-matplotlib/

In [1]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import magenta
import os, time, re
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PLOT_WIDTH = 1200
PLOT_HEIGHT = 800

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

### Setup: read in the music stream

In [23]:
""" Set up music stream """

### for reading in chunks
from collections import deque

def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

def nextChunk(csvStream, chunksize=5):
    return(csvStream.get_chunk(chunksize))

chunksQueue = []
def nextChunkWithOverlap(musStream, cq=chunksQueue, chunksize=5, maxNumChunks=20):
    """
    Iterate over the music stream with rolling window.
    For smoother plotting, set chunksize <<< max number of chunks.
    """
    nextChunk = (musStream.get_chunk(chunksize))
    # make space
    if len(cq) >= maxNumChunks:
        cq.pop(0)
    cq.append(nextChunk)
    #should be sorted always because FIFO but maybe should check.
    res = (pd.concat(cq))
    return(res)

# def hasNext(csvStream):
    

INPUT_PATH = "data_processed/maestro/"
add_input_path = lambda x: "{}/{}".format(INPUT_PATH, x)
MUSIC_STREAM_SUBSTR = "maestro_full_music_stream"

music_files = []
for root, dirs, files in os.walk(INPUT_PATH):
    for file in files:
        if MUSIC_STREAM_SUBSTR in file:
            music_files.append(os.path.join(root, file))

### pick first as the music stream
music_files = sorted(music_files) # play in order
print("Number of music streams found:")
print(len(music_files))
print(music_files[:10])

### should only 1 have file to stream
if (len(music_files) > 1):
    whichStream = int(input("Index (0 ... N-1) of stream to pick:"))
else:
    whichStream = 0
musicStream = csvStream(music_files[whichStream])

Number of music streams found:
1
['data_processed/maestro/maestro_full_music_stream.csv']


In [24]:
nextChunk(musicStream)

,piece,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode
0,piece_0,0,0.00000,0.542394,0.617185,0.94479,"0.0,0.18646,0.19583,0.37083,0.54062,0.69375,0....",0.09687,0.749271,0.718230,1.43021,"0.09687,0.40417,0.70104,0.53854,0.66042,0.7354...",55.0,66.000000,69.0,74.0,"71,55,71,59,62,72,67,74,57,72",44.0,60.300000,58.0,77.0,"60,44,54,55,52,76,56,68,61,77",0.04167,0.206875,0.143750,0.53958,"0.09687,0.21771,0.50521,0.16771,0.11979,0.0416...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
1,piece_0,1,1.00521,1.432576,1.465620,1.98750,"1.00521,1.05625,1.07917,1.23542,1.43021,1.4656...",1.08958,1.704828,1.581250,2.35521,"1.08958,1.42708,1.1875,1.41667,1.49687,1.67083...",59.0,67.909091,67.0,74.0,"74,72,67,66,71,64,72,74,59,62,66",35.0,56.363636,58.0,68.0,"51,60,57,58,68,35,47,68,50,63,63",0.06667,0.272254,0.192710,0.76875,"0.08438,0.37083,0.10833,0.18125,0.06667,0.2052...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
2,piece_0,2,2.15729,2.560937,2.616665,2.90000,"2.15729,2.36042,2.52708,2.70625,2.71458,2.9",2.50938,3.062153,2.903645,4.23542,"4.23542,2.50938,2.71354,2.93333,2.87396,3.10729",59.0,71.333333,72.5,79.0,"67,71,74,78,59,79",58.0,68.333333,69.5,78.0,"58,66,78,77,58,73",0.14896,0.501215,0.196875,2.07812,"2.07812,0.14896,0.18646,0.22708,0.15938,0.20729",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
3,piece_0,3,3.08021,3.500116,3.431250,3.98542,"3.08021,3.08958,3.25625,3.42604,3.43125,3.6041...",3.24062,3.698147,3.612500,4.21458,"3.25,3.24062,3.42812,3.59167,3.6125,3.80521,3....",57.0,69.666667,72.0,79.0,"76,60,79,74,59,79,72,57,71",47.0,64.333333,68.0,76.0,"76,54,68,70,59,63,70,47,72",0.15104,0.198033,0.173960,0.38750,"0.16979,0.15104,0.17188,0.16563,0.18125,0.2010...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
4,piece_0,4,4.17917,4.543751,4.541145,4.92396,"4.17917,4.18542,4.34792,4.54062,4.54167,4.7177...",4.35833,4.792836,4.805730,5.26562,"4.35833,4.52187,4.54271,4.70417,5.26562,4.9072...",55.0,68.125000,70.0,81.0,"72,66,81,72,55,71,69,59",61.0,67.750000,68.5,73.0,"73,61,73,70,67,70,66,62",0.04688,0.249089,0.184375,0.72396,"0.17917,0.33646,0.19479,0.16354,0.72396,0.1895...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...


### Task 0: Stream music audio and display as a webapp

### Task 1: Stream MIDI and display as a webapp

(do audio later after download)

In [16]:
# import jp_proxy_widget
# from scipy.io import wavfile

""" Demo: auto-updating time series plot, use with voila and watch update """

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

""" Set up plot.ly / dash plots to be updated automatically in real-time
"""
### Setup time series plot
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    ### Graph 1: pitch
    dcc.Graph(id='pitch_graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False, value='plasma',
            options=[{'label': c, 'value': c} for c in px.colors.named_colorscales()])
    ]),
    ### Graphs update automatically (separate from the main for-loops for data analysis)
    ### Make sure updates faster than the main for-loops (so don't miss any data updates)
    ### updates every [interval] milliseconds.
    dcc.Interval(id='interval-component', interval=0.5*1000, n_intervals=0)
])

@app.callback(
    Output('pitch_graph', 'figure'), Input("colorscale-dropdown", "value"), Input("interval-component", "n_intervals"))
def update_pitch_figure(colorscale, n=0):
    """
    Update plotly figure. (Like ggplot2: color based on group)
    currBar (global): variable with the current data.
    """
    fig = px.scatter(currBar, x="start_sec", y="pitch_mean", color="pitch_mean",
        color_continuous_scale=colorscale, render_mode="webgl", template="plotly_dark",
        title="Mean pitch per sampled bar (streaming)",range_y=[21, 108]).update_traces(mode='lines+markers')
    return(fig)

# Run app and display result inline in the notebook
app.run_server(mode='inline')

prev_piece = currBar['piece'].tail(1).values[0]
while True:
    currBar = nextChunkWithOverlap(musicStream)
    if currBar is None:
        print(">> End of stream!")
        break
        
    """ Analysis with current bar here """
        
        
    
    ### Take a break between analyses (so plotly can catch up)
    time.sleep(0.1)

KeyboardInterrupt: 

In [ ]:
raise Exception()

### Task 2: Play music and visualize primitive information, construct financial equities, visualize

In [ ]:
""" SPECTROGRAM
    No machine learning required for this.
"""

#

In [ ]:
""" NOTE DENSITY
    No machine learning required for this.
"""

#

### Task 3: Extract music features from MIDI in real-time, construct financial equities, visualize

Sequential learning.
- Validate (try out) against the streamed music audio and series.
- These are the constructed underlyings for financial equities, upon which prediction/regression can work.

Make a local API so other scripts can GET/POST requests (bid/ask) for this.

In [ ]:
""" TEMPO
    Strategy: linear Gaussian state space model / Kalman filter.
    Model tempo (latent variable zt) as a function of notes etc. (observed variables x1 ... xt)
    https://www.researchgate.net/publication/224711190_A_Modified_Kalman_Filtering_Approach_to_On-Line_Musical_Beat_Tracking
"""




In [ ]:
""" HARMONY
"""

#

In [ ]:
""" RHYTHM
"""

#